In [ ]:
import scipy.signal

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.io.wavfile as wavfile

from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, band_pass_filter_complex_signal


In [ ]:
fs = 10000

times = np.linspace(0, 10, 10*fs)
# signal = np.cos(2*np.pi*times*100) + np.cos(2*np.pi*times*250) + np.cos(2*np.pi*times*300)
signal = np.exp(2j*np.pi*times*100) + np.exp(-2j*np.pi*times*100) + np.exp(2j*np.pi*times*250) + np.exp(-2j*np.pi*times*250) + np.exp(2j*np.pi*times*300) \
        + np.exp(2j*np.pi*times*1000) + np.exp(-2j*np.pi*times*1000)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(signal, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
f,m = compute_fft_plot_from_sample_rate(np.real(signal), fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
def compute_complex_bandpass_filter_coefficients(num_coefficients, center_frequency, bandwidth, sample_rate, window="hamming"):
    """
    Inspired by:
    https://dsp.stackexchange.com/questions/41361/how-to-implement-bandpass-filter-on-complex-valued-signal

    It is recommended to use an odd number of coefficients. This creates a linear phase FIR.
    """
    # First compute the coefficients of a FIR low pass filter of the half the given bandwidth. Half
    # the bandwidth is used because it is symmetric around 0.
    filter_coefficients = scipy.signal.firwin(numtaps=num_coefficients, cutoff=bandwidth/2, window=window, fs=sample_rate)

    # Now, we shift the coefficients by the desired frequency.
    filter_shift = center_frequency / sample_rate

    complex_filter_coefficients = []
    for i, f in enumerate(filter_coefficients):
        complex_coeff = f * np.exp(2j * np.pi * filter_shift * i)
        complex_filter_coefficients.append(complex_coeff)
    
    return complex_filter_coefficients


In [ ]:
filter_coefficients = compute_complex_bandpass_filter_coefficients(num_coefficients=101, center_frequency=-1000, bandwidth=50, sample_rate=fs)

w, h = scipy.signal.freqz(filter_coefficients)

fig = go.Figure()
fig.add_scatter(x=w*fs/(2*np.pi),y=np.abs(h))
fig.show()

In [ ]:
filtered_signal = scipy.signal.filtfilt(filter_coefficients, a=1, x=signal)


f,m = compute_fft_plot_from_sample_rate(filtered_signal, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
real_filtered_signal = scipy.signal.filtfilt(filter_coefficients, a=1, x=np.real(signal))
imag_filtered_signal = scipy.signal.filtfilt(filter_coefficients, a=1, x=np.imag(signal))
filtered_signal = real_filtered_signal + 1j * imag_filtered_signal


f,m = compute_fft_plot_from_sample_rate(filtered_signal, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()